In [19]:
import csv
import re
import numpy as np
import pandas as pd
import nltk

# Data Cleaning

In [2]:
%cd

/home/ubuntu


In [3]:
def getTweetData():
    sentiment = []
    tweet = []
    with open('nbs/data/Sentiment-Analysis-Dataset.csv') as text:
        for line in text:
            row = line.split(',')
            sentiment.append(row[1])
            tweet.append(row[3])
        
    del sentiment[0]
    del tweet[0]
    print len(sentiment)
    print len(tweet)
    
    return sentiment, tweet

In [4]:
def cleanTweet(tweet):
    for i in range(0, len(tweet) - 1):
        #lower case and to string
        tweet[i] = str(tweet[i]).lower()
        #links to URL
        tweet[i] = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet[i])
        #username to AT_USER
        tweet[i] = re.sub('@[^\s]+','AT_USER',tweet[i])
        #Remove white spaces
        tweet[i] = re.sub('[\s]+', ' ', tweet[i])
        #replace #word with word
        tweet[i] = re.sub(r'#([^\s]+)', r'\1', tweet[i])
        #trim
        tweet[i] = tweet[i].strip('\'"')
    return tweet

In [5]:
def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in featureList:
        features['contains(%s)' % word] = (word in tweet_words)
    return features

In [6]:
def getStopWordList(fileName):
    stopWords = []
    stopWords.append('AT_USER')
    stopWords.append('URL')
   
    StopWordFile = open(fileName, 'r')
    line = StopWordFile.readline()
    while line:
        word = line.strip()
        stopWords.append(word)
        line = StopWordFile.readline()
    StopWordFile.close()
    return stopWords

In [7]:
def getFeatureVector(tweet, stopWords):
    featureVector = []
    words = str(tweet).split()
    for word in words:
        word = word.strip('\'"?,.')
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", word)
        if(word in stopWords or val is None):
            continue
        else:
            featureVector.append(word.lower())
            
    return featureVector

In [13]:
full_tweets = []
featureList = []

sentiment, tweet = getTweetData()
clean_tweet = cleanTweet(tweet)
stopWords = getStopWordList('nbs/data/stopwords.txt')
for i in range(0, len(clean_tweet) - 1):
    featureVector = getFeatureVector(clean_tweet[i], stopWords)
    featureList.extend(featureVector)
    full_tweets.append((featureVector, sentiment[i]))

1578627
1578627


In [16]:
featureList = list(set(featureList))

In [17]:
len(featureList)

214790

In [15]:
featureList[:10]

['sad',
 'apl',
 'friend',
 'missed',
 'moon',
 'trailer',
 'omg',
 'omgaga',
 'im',
 'sooo']

In [9]:
featureVector

['zzzzzzzzzzzzz', 'meh']

In [11]:
full_tweets[0]

(['sad', 'apl', 'friend'], '0')

# Training Data

In [20]:
training_set = nltk.classify.util.apply_features(extract_features, full_tweets)

Naive Bayes Classifier

In [ ]:
NBClassifier = nltk.NaiveBayesClassifier.train(training_set)

In [ ]:
testTweet = 'Congrats @brodie, this is some COOL analysis'

In [ ]:
processedTestTweet = processTweet(testTweet)
print NBClassifier.classify(extract_features(getFeatureVector(processedTestTweet)))